# 🎯 Objetivo
Este notebook tem como objetivo complementar o nosso notebook principal, focando especificamente nas respostas às perguntas formuladas no início da Etapa 1. Optamos por estruturá-lo separadamente para garantir uma melhor organização e clareza na apresentação das conclusões.

## 📦 Instalação da Biblioteca `nbimporter`

Para importar variáveis, funções ou células de um notebook Jupyter para outro, utilizamos a biblioteca `nbimporter`. 

🔧 **Para instalá-la, execute o seguinte comando no terminal ou em uma célula do Jupyter:**

```bash
pip install nbimporter


In [1]:
import nbimporter
import main
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import ttest_ind
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')


<a id="perguntas"></a>
# ❓ Perguntas e Respostas

A seguir, apresentamos cada pergunta acompanhada de sua resposta final, com base nas análises realizadas ao longo das Etapas 1 a 5 do projeto. Quando aplicável, também incluímos referências às etapas, gráficos, tabelas e demais visualizações que serviram de base para a resposta.

Algumas questões foram abordadas em diferentes momentos do projeto — nesses casos, consolidamos as evidências para oferecer uma visão mais completa.

<a id="1"></a>
### 1 - Existe correlação entre o PIB per capita de um estado e a taxa de mortalidade infantil?

<a id="2"></a>
### 2 - Regiões com maior produção agrícola têm maior PIB per capita?

#### Não, necessariamente. Embora algumas regiões com alta produção agrícola também tenham PIB per capita elevado, isso não é uma regra geral. A análise mostra que a produção agrícola por si só não garante maior desenvolvimento econômico. O que realmente faz diferença é a capacidade de agregar valor à produção, por meio de industrialização, logística eficiente e diversificação econômica.

In [ ]:

df_agricola = pd.read_csv('dataFrame/BrStatsAtualizada.csv', sep=';', low_memory=False)
df_pib_regioes = pd.read_csv('pib_per_capita/pib_per_capita_regioes.csv')  # Dados já agregados por região

# Converter colunas numéricas (tratando formato brasileiro)
if 'VlProducaoAgricola' in df_agricola.columns:
    df_agricola['VlProducaoAgricola'] = df_agricola['VlProducaoAgricola'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)

# Agregar produção agrícola por região e ano (soma)
df_agregado_regiao = df_agricola.groupby(['Ano', 'Regiao'])['VlProducaoAgricola'].sum().reset_index()

# Contar número de municípios por região/ano (para o tamanho dos pontos)
qtd_municipios = df_agricola.groupby(['Ano', 'Regiao'])['Municipio'].nunique().reset_index()
qtd_municipios.rename(columns={'Municipio': 'QtdMunicipios'}, inplace=True)

# Juntar todos os dados (usando a tabela de PIB por regiões)
df_final = pd.merge(df_agregado_regiao, df_pib_regioes, on=['Ano', 'Regiao'])
df_final = pd.merge(df_final, qtd_municipios, on=['Ano', 'Regiao'])

# Converter Ano para string para a animação
df_final['Ano'] = df_final['Ano'].astype(str)

# Criar o gráfico
fig = px.scatter(
    df_final,
    x='PIB_per_capita',  # Coluna da sua tabela de regiões
    y='VlProducaoAgricola',
    color='Regiao',
    size='QtdMunicipios',
    animation_frame='Ano',
    range_x=[0, df_final['PIB_per_capita'].max() * 1.1],
    range_y=[0, df_final['VlProducaoAgricola'].max() * 1.1],
    title='<b>Relação entre PIB per capita e Produção Agrícola por Região (2016-2021)</b>',
    labels={
        'PIB_per_capita': 'PIB per capita (R$)',
        'VlProducaoAgricola': 'Valor da Produção Agrícola (R$)',
        'QtdMunicipios': 'Nº de Municípios',
        'Regiao': 'Região'
    },
    hover_data=['Regiao', 'Ano'],
    height=600,
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_layout(
    font=dict(family="Arial", size=12),
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font_size=20,
    legend_title_text='Região',
    margin=dict(l=40, r=40, t=80, b=40),
    yaxis=dict(gridcolor='lightgrey'),
    xaxis=dict(gridcolor='lightgrey'),
    hovermode='closest'
)

fig.update_traces(
    marker=dict(opacity=0.85, line=dict(width=1, color='DarkSlateGrey'))
)

# Configurações da animação
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500

fig.show()

<a id="3"></a>
### 3 - Cidades com maior área cultivada têm menor densidade populacional? Isso afeta o desenvolvimento econômico?

<a id="4"></a>
### 4 - Cidades mais populosas têm mais empregos, mas piores indicadores de saúde?

<a id="5"></a>
### 5 - Estados com maior volume de exportações têm maior PIB per capita?
#### Não necessariamente. Com base na análise do gráfico de Exportações vs PIB per capita por Estado, é possível perceber que um maior volume de exportações não garante, por si só, um maior PIB per capita. O estado de São Paulo, por exemplo, apresenta um alto volume de exportações, mas não lidera o ranking de PIB per capita. Por outro lado, o Distrito Federal se destaca entre os estados com maior PIB per capita, mesmo com um volume exportador relativamente baixo, o que indica que sua economia está fortemente baseada em setores como administração pública e serviços. Esses exemplos mostram que o desenvolvimento econômico regional depende de múltiplos fatores além do comércio exterior. Entre eles, destacam-se a composição setorial da economia, o nível de diversificação produtiva, a infraestrutura institucional e a presença de políticas públicas que impactam diretamente o rendimento per capita da população.

🔍 Dica: Para visualizar o PIB per capita dos estados de SP e AM, clique na legenda da cor azul (Exportações) no gráfico. Isso ocultará temporariamente as barras de exportações e permitirá ver com mais clareza as barras de PIB per capita desses estados.

In [14]:
df = pd.read_csv("dataFrame/BrStatsAtualizada.csv", sep=";", dtype={"Exportacoes_R$": str})
df.columns = df.columns.str.strip()

# Carregar o dataset de PIB per capita por estado
df_pib_estado = pd.read_csv("pib_per_capita/pib_per_capita_estados.csv")
df_pib_estado["Ano"] = df_pib_estado["Ano"].astype(str)

for col in ["Exportacoes_R$", "QtEmpresas", "PessoalOcupado"]:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(".", "", regex=False).str.replace(",", ".", regex=False),errors="coerce")
    
df["Ano"] = df["Ano"].astype(str)
df_exportacoes = df.groupby(["Ano", "UF"])["Exportacoes_R$"].sum().reset_index()
df_final = pd.merge(df_exportacoes, df_pib_estado, on=["Ano", "UF"])
fig = go.Figure()
anos = sorted(df_final["Ano"].unique())
buttons = []

for i, ano in enumerate(anos):
    df_ano = df_final[df_final["Ano"] == ano].sort_values("PIB_per_capita", ascending=False)
    
    fig.add_trace(go.Bar(
        x=df_ano["UF"],
        y=df_ano["Exportacoes_R$"],
        name="Exportações (R$)",
        marker_color="#1f77b4",
        yaxis="y2",
        visible=(i==0)
    ))

    fig.add_trace(go.Bar(
        x=df_ano["UF"],
        y=df_ano["PIB_per_capita"],
        name="PIB per capita (R$)",
        marker_color="#ff7f0e",
        visible=(i==0)
    ))

    buttons.append(dict(
        label=str(ano),
        method="update",
        args=[{"visible": [False]*len(anos)*2},
              {"title": f"Exportações vs PIB per capita por Estado - {ano}"}]
    ))
    buttons[-1]["args"][0]["visible"][i*2] = True
    buttons[-1]["args"][0]["visible"][i*2 + 1] = True

fig.update_layout(
    title="Exportações vs PIB per capita por Estado",
    xaxis=dict(title="Estados", tickangle=45),
    yaxis=dict(title="PIB per capita (R$)", side="left"),
    yaxis2=dict(
        title="Exportações (R$)",
        overlaying="y",
        side="right"
        
    ),
    updatemenus=[{
        "buttons": buttons,
        "direction": "down",
        "showactive": True,
        "x": 1.0,
        "y": 1.15
    }],
    barmode="group",
    plot_bgcolor="white",
    height=600,
    width=1000,
    legend=dict(x=1.1, y=1)
)

fig.show()



<a id="6"></a>
### 6 - Cidades com maior produção agrícola têm melhores ou piores indicadores de saúde infantil?

<a id="7"></a>
### 7 - Regiões com maior produção pecuária têm maior PIB per capita?

<a id="8"></a>
### 8 - Cidades com maior densidade populacional têm maior volume de exportações?
#### Sim. Embora o modelo de regressão linear tenha apresentado baixo poder preditivo (com R² de apenas 6,39%), os resultados apontam que a densidade populacional possui um coeficiente positivo elevado, indicando uma tendência de que estados mais densamente povoados tendem a exportar mais. Ou seja, mesmo com limitações no ajuste global do modelo, há indícios de que a densidade populacional exerce alguma influência positiva sobre o volume de exportações. Esse padrão sugere que fatores como concentração urbana, infraestrutura e logística podem contribuir para o desempenho exportador dos estados.

In [6]:
# Carregar o dataset
df = pd.read_csv("dataFrame/BrStatsAtualizada.csv", sep=";")

# Converter as colunas relevantes de string para float (substituir vírgula por ponto)
for col in ["Exportacoes_R$", "povoamento", "QtEmpresas"]:
    df[col] = df[col].astype(str).str.replace(",", ".").astype(float)

# Selecionar as variáveis relevantes
df_model = df[["Exportacoes_R$", "povoamento", "QtEmpresas"]].copy()

# Remover registros com valores faltantes
df_model.dropna(inplace=True)

# Definir variáveis explicativas (X) e variável alvo (y)
X = df_model[["povoamento", "QtEmpresas"]]
y = df_model["Exportacoes_R$"]

# Padronizar as variáveis explicativas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir em conjuntos de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# Criar o modelo de regressão linear
model = LinearRegression()

# Treinar o modelo com os dados de treino
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Obter coeficientes e intercepto
coeficientes = model.coef_
intercepto = model.intercept_

# Calcular métricas de avaliação
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Mostrar os coeficientes com os nomes das variáveis
for nome, coef in zip(["Densidade Populacional (povoamento)", "Número de Empresas (QtEmpresas)"], coeficientes):
    print(f"{nome}: {coef:.4f}")

print(f"\nIntercepto: {intercepto:.2f}")
print(f"Erro Quadrático Médio (MSE): {mse:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2%}")


Densidade Populacional (povoamento): 602281.6500
Número de Empresas (QtEmpresas): -647206.8851

Intercepto: 461016291.49
Erro Quadrático Médio (MSE): 12273326486112360448.00
Coeficiente de Determinação (R²): 6.39%


<a id="9"></a>
### 9 - Estados com maior informalidade têm menor PIB?

<a id="10"></a>
### 10 - Quais são as principais diferenças socioeconômicas entre as regiões do Brasil?